### Cost SQL Script

In [ ]:
ALTER TABLE sourcing_request_items_update ADD COLUMN delivery_mode VARCHAR;
ALTER TABLE sourcing_request_items_update ADD COLUMN status VARCHAR;

-- Step 2: Update the sourcing_request_items table with data from the quotation table
UPDATE sourcing_request_items_update sri
SET
    delivery_mode = q.delivery_mode,
    status = q.status
FROM quotations q
WHERE sri.sourcing_request_id = q.sourcing_request_id

BEGIN;

-- Step 1: Add columns to the sourcing_request_items table if they do not exist
DO $$
BEGIN
    IF NOT EXISTS (SELECT 1 FROM information_schema.columns WHERE table_name='sourcing_request_items' AND column_name='delivery_mode') THEN
        ALTER TABLE sourcing_request_items ADD COLUMN delivery_mode VARCHAR;
    END IF;
    IF NOT EXISTS (SELECT 1 FROM information_schema.columns WHERE table_name='sourcing_request_items' AND column_name='status') THEN
        ALTER TABLE sourcing_request_items ADD COLUMN status VARCHAR;
    END IF;
END;
$$;

ALTER TABLE aggregated_sourcing_request_costs ADD COLUMN delivery_mode VARCHAR;

UPDATE aggregated_sourcing_request_costs sri
SET delivery_mode = q.delivery_mode
FROM quotations q
WHERE sri.sourcing_request_id = q.sourcing_request_id;

-- Commit the transaction
COMMIT;

ALTER TABLE sourcing_request_items_update
	drop column delivery_mode;

CREATE TABLE aggregated_sourcing_request_costs AS
SELECT 
    sourcing_request_id,
    SUM(total_price_exfactory) AS total_price_exfactory,
    SUM(total_cost_outbound_logistics_air) AS total_cost_outbound_logistics_air,
    SUM(total_cost_outbound_logistics_lcl_ddp) AS total_cost_outbound_logistics_lcl_ddp,
    SUM(total_cost_air) AS total_cost_air,
    SUM(total_cost_lcl_ddp) AS total_cost_lcl_ddp,
    SUM(total_cost_fcl_ddp) AS total_cost_fcl_ddp
FROM 
    sourcing_request_items_update
GROUP BY 
    sourcing_request_id;
   
   UPDATE aggregated_sourcing_request_costs sri
SET
    delivery_mode = q.delivery_mode,
    status = q.status
FROM quotation q
WHERE sri.sourcing_request_id = q.sourcing_request_id;

DELETE FROM aggregated_sourcing_request_costs
WHERE sourcing_request_id = 22;

create table new_aggregated_sourcing_request_costs AS
	select * from public.aggregated_sourcing_request_costs;

DELETE FROM aggregated_sourcing_request_costs
WHERE sourcing_request_id in (1, 3, 7, 8, 9, 30, 33, 53, 58, 59, 61, 63, 64, 73, 87, 90, 93, 95, 101, 106, 112, 116);

ALTER TABLE aggregated_sourcing_request_costs ADD COLUMN total_costs float8;


CREATE TABLE aggregated_sourcing_request_costs_1 AS
SELECT 
    sourcing_request_id,
    SUM(total_price_exfactory) AS total_price_exfactory,
    SUM(total_cost_outbound_logistics_air) AS total_cost_outbound_logistics_air,
    SUM(total_cost_outbound_logistics_lcl_ddp) AS total_cost_outbound_logistics_lcl_ddp,
    SUM(total_cost_outbound_logistics_fcl_ddp) AS total_cost_outbound_logistics_fcl_ddp,
    SUM(total_cost_air) AS total_cost_air,
    SUM(total_cost_lcl_ddp) AS total_cost_lcl_ddp,
    SUM(total_cost_fcl_ddp) AS total_cost_fcl_ddp
FROM 
    sourcing_request_items_update
GROUP BY 
    sourcing_request_id;

ALTER TABLE aggregated_sourcing_request_costs_1 ADD COLUMN delivery_mode VARCHAR;
   
  UPDATE aggregated_sourcing_request_costs_1 sri
SET delivery_mode = q.delivery_mode
FROM quotations q
WHERE sri.sourcing_request_id = q.sourcing_request_id;

DELETE FROM aggregated_sourcing_request_costs_1
WHERE sourcing_request_id = 22;

ALTER TABLE aggregated_sourcing_request_costs_1 ADD COLUMN total_costs float8;

UPDATE aggregated_sourcing_request_costs_1
SET total_costs = CASE 
    WHEN delivery_mode = '{AIR,LCL_DDP}' THEN 
        total_price_exfactory + total_cost_outbound_logistics_air + total_cost_outbound_logistics_lcl_ddp
    WHEN delivery_mode = '{AIR}' THEN 
        total_price_exfactory + total_cost_outbound_logistics_air
    WHEN delivery_mode = '{LCL_DDP}' THEN 
        total_price_exfactory + total_cost_outbound_logistics_lcl_ddp
    WHEN delivery_mode = '{LCL_DDP,FCL_DDP}' THEN 
        total_price_exfactory + total_cost_outbound_logistics_lcl_ddp + total_cost_outbound_logistics_fcl_ddp
    WHEN delivery_mode = '{AIR,LCL_DDP,FCL_DDP}' THEN 
        total_price_exfactory + total_cost_outbound_logistics_air + total_cost_outbound_logistics_lcl_ddp + total_cost_outbound_logistics_fcl_ddp
    ELSE 
        NULL
END;

ALTER TABLE aggregated_sourcing_request_costs_1 ADD COLUMN customer_id INT;
   
UPDATE aggregated_sourcing_request_costs_1 sri
SET customer_id = c.customer_id
FROM customers_readonly c
WHERE sri.sourcing_request_id = c.sourcing_request_id;

DELETE FROM aggregated_sourcing_request_costs_1
WHERE sourcing_request_id = 22;

### SKU SQL Script

In [ ]:
CREATE TABLE TEST_skus1 (
  quotation_id int4 NULL,
  quotation_year int4 NULL,
  quotation_month int4 NULL,
  quotation_date int4 NULL,
  quotation_type varchar(50000) NULL,
  status varchar(50000) NULL,
  sourcing_request_item_id int4 NULL,
  quantity int4 NULL,
  customer_id int4 NULL,
  country varchar(255) NULL,
  country_name varchar(50000) NULL,
  region varchar(50000) NULL,
  subcategory varchar(255) NULL,
  product_id int4 NULL,
  scrape_product_id int4 NULL,
  taxonomy_id int4 NULL,
  category_level_1 varchar(255) NULL,
  category_level_2 varchar(255) NULL,
  category_level_3 varchar(255) NULL
);

INSERT INTO TEST_skus1 (
  quotation_id,
  quotation_year,
  quotation_month,
  quotation_date,
  quotation_type,
  status,
  sourcing_request_item_id,
  quantity,
  customer_id,
  country,
  country_name,
  region,
  subcategory,
  product_id,
  scrape_product_id,
  taxonomy_id,
  category_level_1,
  category_level_2,
  category_level_3
)
SELECT
  q.quotation_id,
  CAST(SUBSTRING(q.quotation_date, 1, 4) AS INT) AS quotation_year,
  CAST(SUBSTRING(q.quotation_date, 6, 2) AS INT) AS quotation_month,
  CAST(SUBSTRING(q.quotation_date, 9, 2) AS INT) AS quotation_date,
  q.quotation_type,
  q.status,
  i.sourcing_request_item_id,
  s.quantity,
  s.customer_id,
  c.country,
  r.country_name,
  r.region,
  c.subcategory,
  s.product_id,
  p.scrape_product_id,
  p.taxonomy_id,
  t.category_level_1,
  t.category_level_2,
  t.category_level_3
FROM quotations q
LEFT JOIN quotation_items i ON q.quotation_id = i.quotation_id
LEFT JOIN sourcing_request_items s ON i.sourcing_request_item_id = s.sourcing_request_item_id
LEFT JOIN customers_readonly c ON s.customer_id = c.customer_id
RIGHT JOIN countries r ON c.country = r.country
LEFT JOIN products p ON s.product_id = p.product_id
LEFT JOIN taxonomies t ON p.taxonomy_id = t.taxonomy_id;